In [155]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime



In [156]:
url = 'https://eventsmadeira.com/en/event-listing/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')
divs = soup.find_all('div', class_='gt-event-style-3')
len(divs)


78

In [184]:
def get_data_text(result):
    list_items_text = [li.text for li in result.find_all('li')]
    if list_items_text and len(list_items_text) > 1:
        # remove duplicities
        return ', '.join(list(set(list_items_text)))
    return result.text

def fill_datetime(info_container, data):
    datetime_labels = [('start_date', 'start_time', 'gt-start-date'), ('end_date', 'end_time', 'gt-end-date')]
    for date_label, time_label, class_name in datetime_labels:
        datetime_scrapped = info_container.find('li', {'class': class_name})

        if datetime_scrapped is not None:
            datetime_ = datetime_scrapped.find('div', {'class': 'gt-inner'}).text
            print(datetime_)
            datetime_obj = datetime.strptime(datetime_, '%d/%m/%Y %H:%M') if len(datetime_.split(' ')) > 0 else datetime.strptime(datetime_, '%d/%m/%Y')

            data[date_label].append(datetime_obj.date())
            data[time_label].append(datetime_obj.time())

        else:
            data[date_label].append('-')
            data[time_label].append('-')

def fill_description(page_content, data):
    description = page_content.find('div', {'class': 'gt-content'})
    data['description'].append(' '.join([p.text for p in description]).replace('\n', '') if description else '-')

def fill_event_data(url, data):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    page_content = soup.find('div', {'class': 'gt-page-content'})
    info_container = soup.find('div', {'class': 'gt-content-detail-box'})
    if not info_container:
        return None
    else: 
        fill_datetime(info_container, data)
        fill_description(page_content, data)
        address = info_container.find('li', {'class': 'gt-address'}).find('div', {'class': 'gt-inner'}).text if info_container.find('li', {'class': 'gt-address'}) else '-'
        # todo: finish location + categories + title
        location = info_container.find('li', {'class': 'gt-locations'}).find('div', {'class': 'gt-inner'}) if info_container.find('li', {'class': 'gt-locations'}) else '-'
        
        print(location)
        data['address'].append(address)
        data['location'].append(location)


        
        
    

    


In [186]:
all_fields = ["title", "start_date", "end_date", "start_time", "end_time",  "location", "category", 'address', 'description', 'sourceLink', 'sourceId']
fields = ["title", "location", "category"]

data = {field: [] for field in all_fields}

for div in divs[6:7]:
    print(url)
    url = div.find('div', class_='gt-title').find('a')['href']
    data['sourceLink'].append(url)
    data['sourceId'].append(1)
    fill_event_data(url, data)


https://eventsmadeira.com/en/event/mandolin-orchestra-weekly-concerts/
17/11/2023 21:00
01/12/2024 22:00
<div class="gt-inner"><ul><li><a href="https://eventsmadeira.com/en/location/funchal-en/?post_type=event">Funchal</a></li><li><a href="https://eventsmadeira.com/en/location/funchal-en/?post_type=event">Funchal</a></li></ul></div>


In [159]:
# fields = ["start_date", "end_date", "start_time", "end_time"]
# # todo: do not introduce new fields, but modify the existing ones
# data_dt = {field: [] for field in fields}

# for date, time in zip(data['date'], data['time']): 
#     splitted_date = date.split(' - ')
#     if len(splitted_date) > 1:
#         # todo: handle case where time is interval (e.g. 10:00 - 12:00)
#         start = datetime.strptime(splitted_date[0] + time, '%d/%m/%Y%H:%M')
#         end = datetime.strptime(splitted_date[1], '%d/%m/%Y')
#         data_dt['start_date'].append(start.date())
#         data_dt['end_date'].append(end.date())
#         data_dt['start_time'].append(start.time())
#         data_dt['end_time'].append(end.time())
#     else:
#         if splitted_date[0] == '-': 
#             data_dt['start_date'].append('-')
#             data_dt['end_date'].append('-')
#             data_dt['start_time'].append('-')
#             data_dt['end_time'].append('-')
#         else: 
#             start = datetime.strptime(splitted_date[0], '%d/%m/%Y')
#             data_dt['start_date'].append(start.date())
#             data_dt['end_date'].append('-')
#             data_dt['start_time'].append(start.time())
#             data_dt['end_time'].append('-')


        


In [181]:
# df = pd.DataFrame(data)

data


{'title': [],
 'start_date': [datetime.date(2023, 11, 17)],
 'end_date': [datetime.date(2024, 12, 1)],
 'start_time': [datetime.time(21, 0)],
 'end_time': [datetime.time(22, 0)],
 'location': ['FunchalFunchal'],
 'category': [],
 'address': ['Assembleia Legislativa Regional '],
 'description': [' Recognized as the oldest and youngest Mandolin Orchestra in Europe, a distinction of relevance in a region of excellence within the European community, the Madeira Mandolin Orchestra® presents itself in the 2023/2024 Musical Season with a unique artistic proposal and diversified, contemplating programs adapted to the mandolin of the great classics of world music, striving for the quality that is recognized in the preservation and dissemination of the history of the mandolin in Madeira.  Hear, feel and communicate the universal language of music.  For more information about the Orchestra and to acquire tickets for the concerts, visit the official website – www.madeiramandolin.com.  Concerts:Nov